In [ ]:
from eose.instruments import  CircularGeometry, RectangularGeometry, BasicSensor

from pydantic import ValidationError

# Example usage of SphericalGeometry class used to define the field of view of the instrument

circ_geom = CircularGeometry(diameter=90.0)
print(circ_geom)

rect_geom = RectangularGeometry(angle_height= 90.0, angle_width= 45)
print(rect_geom)


In [ ]:
# Example simple usage of eose-api BasicSensor class

sensor = BasicSensor(   mass= 100.5,
                        volume= 0.75,
                        power= 150.0,
                        field_of_view = CircularGeometry(diameter=30.0),
                        data_rate= 10.5,
                        bits_per_pixel= 16
                    )

print(sensor)

sensor = BasicSensor(   mass= 100.5,
                        volume= 0.75,
                        power= 150.0,
                        orientation= list([0.5,0.5,0.5,0.5]),
                        field_of_view = RectangularGeometry(ngle_width=30.0, angle_height=10.0),
                        data_rate= 50.5,
                        bits_per_pixel= 8
                    )

print(sensor)

sensor = BasicSensor(   mass= 100.5,
                    volume= 0.75,
                    power= 150.0,
                    orientation= list([0.5,0.5,0.5,0.5]),
                    data_rate= 50.5,
                    bits_per_pixel= 8
                )

print(sensor)

display(sensor.model_dump_json())

In [ ]:
%reset -f

## Example usage with TAT-C and InstruPy

In [1]:
import json
from datetime import datetime, timedelta, timezone

from tatc.schemas import Instrument as TATC_Instrument, Satellite as TATC_Satellite, TwoLineElements, Point
from tatc.analysis import collect_orbit_track, OrbitCoordinate, OrbitOutput
from tatc.analysis import (
    collect_multi_observations,
    aggregate_observations,
    reduce_observations,
)

from instrupy.basic_sensor_model import BasicSensorModel as InstruPy_BasicSensorModel

from eose.instruments import CircularGeometry, BasicSensor
from eose.datametric import DataMetricRequest, BasicSensorDataMetricInstantaneous, BasicSensorDataMetricSample, BasicSensorDataMetricRecord, BasicSensorDataMetricResponse
from eose.propagation import PropagationRecord, PropagationRequest, PropagationResponse
from eose.orbits import GeneralPerturbationsOrbitState
from eose.utils import CartesianReferenceFrame, FixedOrientation
from eose.coverage import (
    CoverageRecord,
    CoverageRequest,
    CoverageResponse,
    CoverageSample,
)
from eose.grids import UniformAngularGrid
from eose.satellites import Satellite

from shapely.geometry import box, mapping
from joblib import Parallel, delayed
from scipy.stats import hmean
import pandas as pd

from astropy.time import Time as AstroPy_Time

pd.set_option('display.max_rows', None)

In [2]:
# define the orbit and the instrument
iss_omm_str = '[{"OBJECT_NAME":"ISS (ZARYA)","OBJECT_ID":"1998-067A","EPOCH":"2024-06-07T09:53:34.728000","MEAN_MOTION":15.50975122,"ECCENTRICITY":0.0005669,"INCLINATION":51.6419,"RA_OF_ASC_NODE":3.7199,"ARG_OF_PERICENTER":284.672,"MEAN_ANOMALY":139.0837,"EPHEMERIS_TYPE":0,"CLASSIFICATION_TYPE":"U","NORAD_CAT_ID":25544,"ELEMENT_SET_NO":999,"REV_AT_EPOCH":45703,"BSTAR":0.00033759,"MEAN_MOTION_DOT":0.00019541,"MEAN_MOTION_DDOT":0}]'
iss_omm = json.loads(iss_omm_str)[0]

basic_sensor = BasicSensor(   mass= 100.5,
                        volume= 0.75,
                        power= 150.0,
                        field_of_view = CircularGeometry(diameter=60.0),
                        data_rate= 10.5,
                        bits_per_pixel= 16
                    )

mission_start = datetime(2024, 1, 1, tzinfo=timezone.utc)
mission_duration = timedelta(hours=2)
propagate_time_step = timedelta(minutes=1)

In [3]:
# Run propagation with TAT-C to get satellite states
def propagate_tatc(request: PropagationRequest) -> PropagationResponse:
    return PropagationResponse(
        records=collect_orbit_track(
            TATC_Satellite(
                name=request.orbit.object_name,
                orbit=TwoLineElements(tle=request.orbit.to_tle()),
            ),
            TATC_Instrument(name="Instrument"),
            pd.date_range(
                request.start, request.start + request.duration, freq=request.time_step
            ),
            coordinates=(
                OrbitCoordinate.ECI
                if request.frame == CartesianReferenceFrame.ICRF
                else OrbitCoordinate.ECEF
            ),
            orbit_output=OrbitOutput.POSITION_VELOCITY,
        ).apply(
            lambda r: PropagationRecord(
                time=r.time,
                frame=request.frame,
                position=r.geometry.coords[0],
                velocity=r.velocity.coords[0],
                body_orientation=FixedOrientation.NADIR_GEOCENTRIC,
            ),
            axis=1,
        ),
    )



request = PropagationRequest(
    orbit=GeneralPerturbationsOrbitState.from_omm(iss_omm),
    start=mission_start,
    duration=mission_duration,
    time_step=propagate_time_step,
    frame=CartesianReferenceFrame.ICRF,
)

display(request.model_dump_json())

propagation_response = propagate_tatc(request)

display(propagation_response.model_dump_json())

propagation_data = propagation_response.as_dataframe()

display(propagation_data)


'{"orbit":{"object_name":"ISS (ZARYA)","object_id":"1998-067A","epoch":"2024-06-07T09:53:34.728000","mean_motion":15.50975122,"eccentricity":0.0005669,"inclination":51.6419,"ra_of_asc_node":3.7199,"arg_of_pericenter":284.672,"mean_anomaly":139.0837,"ephemeris_type":0,"classification_type":"U","norad_cat_id":25544,"element_set_no":999,"rev_at_epoch":45703,"bstar":0.00033759,"mean_motion_dot":0.00019541,"mean_motion_ddot":0.0},"start":"2024-01-01T00:00:00Z","duration":"PT2H","time_step":"PT1M","frame":"ICRF"}'

'{"records":[{"frame":"ICRF","time":"2024-01-01T00:00:00Z","position":[-4499686.1202688,-232613.21550674032,5091657.311614182],"velocity":[-1677.6634157866658,-7252.659671818871,-1804.8077999287968],"body_orientation":"NADIR_GEOCENTRIC","view_orientation":[0.0,0.0,0.0,1.0]},{"frame":"ICRF","time":"2024-01-01T00:01:00Z","position":[-4590028.087700935,-666915.9700864203,4971828.475405545],"velocity":[-1332.576798616102,-7218.509492576278,-2187.9548996394055],"body_orientation":"NADIR_GEOCENTRIC","view_orientation":[0.0,0.0,0.0,1.0]},{"frame":"ICRF","time":"2024-01-01T00:02:00Z","position":[-4659474.854577173,-1098182.7987153332,4829301.789743176],"velocity":[-981.4248032269486,-7151.498310442072,-2561.1120354711065],"body_orientation":"NADIR_GEOCENTRIC","view_orientation":[0.0,0.0,0.0,1.0]},{"frame":"ICRF","time":"2024-01-01T00:03:00Z","position":[-4707710.266126298,-1524450.42817976,4664727.924286885],"velocity":[-625.8040584505688,-7051.930817092407,-2922.5776733547577],"body_orientati

,geometry,frame,time,position,velocity,body_orientation,view_orientation
0,POINT Z (83.10851 48.53978 432820.67293),ICRF,2024-01-01 00:00:00+00:00,"[-4499686.1202688, -232613.21550674032, 509165...","[-1677.6634157866658, -7252.659671818871, -180...",NADIR_GEOCENTRIC,"[0, 0, 0, 1]"
1,POINT Z (88.15274 47.03603 432699.62950),ICRF,2024-01-01 00:01:00+00:00,"[-4590028.087700935, -666915.9700864203, 49718...","[-1332.576798616102, -7218.509492576278, -2187...",NADIR_GEOCENTRIC,"[0, 0, 0, 1]"
2,POINT Z (92.88653 45.30148 432541.96420),ICRF,2024-01-01 00:02:00+00:00,"[-4659474.854577173, -1098182.7987153332, 4829...","[-981.4248032269486, -7151.498310442072, -2561...",NADIR_GEOCENTRIC,"[0, 0, 0, 1]"
3,POINT Z (97.30894 43.36259 432355.86862),ICRF,2024-01-01 00:03:00+00:00,"[-4707710.266126298, -1524450.42817976, 466472...","[-625.8040584505688, -7051.930817092407, -2922...",NADIR_GEOCENTRIC,"[0, 0, 0, 1]"
4,POINT Z (101.43038 41.24458 432150.55101),ICRF,2024-01-01 00:04:00+00:00,"[-4734514.636033979, -1943778.3089340637, 4478...","[-267.3323844160795, -6920.2597150227775, -327...",NADIR_GEOCENTRIC,"[0, 0, 0, 1]"
5,POINT Z (105.26888 38.97076 431936.04445),ICRF,2024-01-01 00:05:00+00:00,"[-4739765.803263393, -2354257.448934487, 42725...","[92.35841105772057, -6757.084038207287, -3603....",NADIR_GEOCENTRIC,"[0, 0, 0, 1]"
6,POINT Z (108.84696 36.56215 431722.99742),ICRF,2024-01-01 00:06:00+00:00,"[-4723439.752729724, -2754019.127748536, 40467...","[451.62986483811886, -6563.146823572038, -3920...",NADIR_GEOCENTRIC,"[0, 0, 0, 1]"
7,POINT Z (112.18929 34.03746 431522.45036),ICRF,2024-01-01 00:07:00+00:00,"[-4685610.792771687, -3141243.4513263768, 3802...","[808.844293869679, -6339.332120422664, -4219.4...",NADIR_GEOCENTRIC,"[0, 0, 0, 1]"
8,POINT Z (115.32100 31.41320 431345.60236),ICRF,2024-01-01 00:08:00+00:00,"[-4626451.284063706, -3514167.7073642504, 3540...","[1162.372303991752, -6086.661327825185, -4499....",NADIR_GEOCENTRIC,"[0, 0, 0, 1]"
9,POINT Z (118.26669 28.70385 431203.57224),ICRF,2024-01-01 00:09:00+00:00,"[-4546230.916459875, -3871094.48084614, 326294...","[1510.6003386121554, -5806.288853847601, -4758...",NADIR_GEOCENTRIC,"[0, 0, 0, 1]"


In [4]:
# Run coverage analysis with TAT-C to get access-times at Target ground points. It is assumed that the sensor shall be of Circular FOV geometry.


def coverage_tatc(request: CoverageRequest) -> CoverageResponse:
    unique_ids = len(set(target.id for target in request.targets)) == len(
        request.targets
    )
    points = [
        Point(
            id=target.id if unique_ids and isinstance(target.id, int) else i,
            longitude=target.position[0],
            latitude=target.position[1],
            altitude=target.position[2] if len(target.position) > 2 else 0,
        )
        for i, target in enumerate(request.targets)
    ]
    satellites = [
        TATC_Satellite(
            name=satellite.orbit.object_name,
            orbit=TwoLineElements(tle=satellite.orbit.to_tle()),
            instruments=[
                TATC_Instrument(name="Default", field_of_regard=satellite.field_of_view)
            ],
        )
        for satellite in request.satellites
    ]

    aggregated_obs = aggregate_observations(
        pd.concat(
            Parallel(-1)(
                delayed(collect_multi_observations)(
                    point,
                    satellites,
                    request.start,
                    request.start + request.duration,
                )
                for point in points
            )
        )
    )
    reduced_obs = reduce_observations(aggregated_obs)

    records = list(
        reduced_obs.apply(
            lambda r: CoverageRecord(
                target=request.targets[
                    points.index(next(p for p in points if p.id == r["point_id"]))
                ],
                samples=aggregated_obs[aggregated_obs.point_id == r["point_id"]].apply(
                    lambda s: CoverageSample(start=s.start, duration=s.end - s.start),
                    axis=1,
                ),
                mean_revisit=(
                    None
                    if pd.isnull(r["revisit"])
                    else timedelta(seconds=r["revisit"].total_seconds())
                ),
                number_samples=r["samples"],
            ),
            axis=1,
        )
    ) + [
        CoverageRecord(target=request.targets[i], mean_revisit=None, number_samples=0)
        for i, point in enumerate(points)
        if not any(reduced_obs["point_id"] == point.id)
    ]
    records.sort(key=lambda r: r.target.id)

    return CoverageResponse(
        records=records,
        harmonic_mean_revisit=(
            None
            if reduced_obs.dropna(subset="revisit").empty
            else timedelta(
                seconds=hmean(
                    reduced_obs.dropna(subset="revisit")["revisit"].dt.total_seconds()
                )
            )
        ),
        coverage_fraction=len(reduced_obs.index) / len(points),
    )



request = CoverageRequest(
    satellites=[
        Satellite(
            orbit=GeneralPerturbationsOrbitState.from_omm(iss_omm), field_of_view=basic_sensor.field_of_view.diameter
        )
    ],
    targets=UniformAngularGrid(
        delta_latitude=20, delta_longitude=20, region=mapping(box(-180, -50, 180, 50))
    ).as_targets(),
    start=mission_start,
    duration=mission_duration,
)

#display(request.model_dump_json())

coverage_response = coverage_tatc(request)

#display(coverage_response.model_dump_json())

coverage_data = coverage_response.as_dataframe()

display(coverage_data)

,geometry,target,samples,mean_revisit,number_samples
0,POINT (-170.00000 -40.00000),"{'id': 36, 'crs': None, 'position': [-170.0, -...",[],NaT,0
1,POINT (-150.00000 -40.00000),"{'id': 37, 'crs': None, 'position': [-150.0, -...",[],NaT,0
2,POINT (-130.00000 -40.00000),"{'id': 38, 'crs': None, 'position': [-130.0, -...",[],NaT,0
3,POINT (-110.00000 -40.00000),"{'id': 39, 'crs': None, 'position': [-110.0, -...",[],NaT,0
4,POINT (-90.00000 -40.00000),"{'id': 40, 'crs': None, 'position': [-90.0, -4...","[{'start': '2024-01-01T00:50:17.988110Z', 'dur...",NaT,1
5,POINT (-70.00000 -40.00000),"{'id': 41, 'crs': None, 'position': [-70.0, -4...",[],NaT,0
6,POINT (-50.00000 -40.00000),"{'id': 42, 'crs': None, 'position': [-50.0, -4...",[],NaT,0
7,POINT (-30.00000 -40.00000),"{'id': 43, 'crs': None, 'position': [-30.0, -4...",[],NaT,0
8,POINT (-10.00000 -40.00000),"{'id': 44, 'crs': None, 'position': [-10.0, -4...",[],NaT,0
9,POINT (10.00000 -40.00000),"{'id': 45, 'crs': None, 'position': [10.0, -40...",[],NaT,0


In [10]:
def datametrics_instrupy(request: DataMetricRequest) -> BasicSensorDataMetricResponse:
    
    if request.sensor.type == "BasicSensor":
        basic_sensor = request.sensor
    else:
        raise ValueError(f"Expected type 'BasicSensor', but got {sensor.type}.")

    coverage_response = request.coverage_response
    prop_response = request.propagation_response
    datametric_start = request.start
    datametric_duration = request.duration
    
    # define the InstruPy basic sensor model
    # TODO: accept a general pointing orientation
    instrupy_basic_sensor = InstruPy_BasicSensorModel.from_dict({"orientation": {"referenceFrame": "SC_BODY_FIXED", "convention": "REF_FRAME_ALIGNED"},
                                             "fieldOfViewGeometry": {"shape": "CIRCULAR", "diameter": basic_sensor.field_of_view.diameter}
                                                                })
    
    def propagation_records_within_time_range(propagate_response, start_time, stop_time):
        """ Return propagation records within the specified time range. """
        #print("Start time and end date are: ", start_time, end_time)
        # iterate through all the records, and store the records corresponding to the given time range
        filtered_propagate_records = []
        for propagate_record in propagate_response.records:
                    
            # Check if the time falls within the range
            if start_time <= propagate_record.time <= stop_time:
                filtered_propagate_records.append(propagate_record)
                #print(record.time)
    
        return filtered_propagate_records
    
    bs_record = [] # aggregation of data-metrics across multple targets, and multiple overpasses for each target.
    # iterate through the coverage records
    for cov_record in coverage_response.records:
        # each record consists of coverage samples (start, duration) for a target point. It is assumed that the record is unique for a target point.
        #print(record, "\n")
        # iterate through each coverage sample within a coverage record
        target = cov_record.target
        #print("target = ", target, "\n")
        bs_sample = [] # aggregation of data-metrics over a multiple overpasses for the target
        for cov_sample in cov_record.samples:
            #print("\n coverage record sample")
            #print(sample.start, sample.duration, "\n")
    
            # from the propagation response, find satellite states within the coverage sample period
            _start = max(datametric_start, cov_sample.start)
            _stop = min(datametric_start+datametric_duration, cov_sample.start+cov_sample.duration)
            pr = propagation_records_within_time_range(prop_response, _start, _stop)
    
            #print("Propagate state: ", pr)
    
            bs_i = [] # aggregation of data-metrics over a single overpass (=coverage sample)
            if pr is not []: # if propagation states are available within the time-period
                
                # iterate through the propagation states   
                for _pr in pr: 
                    # get the time in Julian Date UT1
                    time_utc = AstroPy_Time(_pr.time.astimezone(timezone.utc), scale='utc') # Convert to astropy Time object
                    time_ut1 = time_utc.ut1  # Convert to UT1 time scale
                    jd_ut1 = time_ut1.jd  # Get the Julian Date
            
                    # calculate the corresponding instantaneous data metrics
                    SpacecraftOrbitState = {'time [JDUT1]':jd_ut1, 'x [km]': _pr.position[0]*1e-3, 'y [km]': _pr.position[1]*1e-3, 'z [km]': _pr.position[2]*1e-3, 
                                            'vx [km/s]': _pr.velocity[0]*1e-3, 'vy [km/s]': _pr.velocity[1]*1e-3, 'vz [km/s]': _pr.velocity[2]*1e-3}
                    TargetCoords = {'lat [deg]': target.position[1], 'lon [deg]': target.position[0]}
                    obsv_metrics = instrupy_basic_sensor.calc_data_metrics(SpacecraftOrbitState, TargetCoords)
                    #print(obsv_metrics)
            
                    # create the instantaneous data metric object
                    _bs_i = BasicSensorDataMetricInstantaneous(time=_pr.time, 
                                                                incidence_angle=obsv_metrics["incidence angle [deg]"],
                                                                look_angle=obsv_metrics["look angle [deg]"],
                                                                observation_range=obsv_metrics["observation range [km]"],
                                                                solar_zenith=obsv_metrics["solar zenith [deg]"],
                                                              )
                    bs_i.append(_bs_i)
    
            # create the datametrics sample which is an aggregation of datametrics instances for a target at a single overpass
            _bs_sample = BasicSensorDataMetricSample(instantaneous_metrics=bs_i)
            bs_sample.append(_bs_sample)
    
        # create the datametrics record which is an aggregation of datametrics samples for a target (across multiple overpasses)
        _bs_record = BasicSensorDataMetricRecord(target=target, samples=bs_sample)
        bs_record.append(_bs_record)
    
    # create the datemetrics response
    datametrics_response = BasicSensorDataMetricResponse(records=bs_record)
    #display(datametrics_response.model_dump_json())

    return datametrics_response


In [11]:
request = DataMetricRequest(
    sensor=basic_sensor,
    start=mission_start,
    duration=mission_duration,
    propagation_response=propagation_response,
    coverage_response=coverage_response
)

#display(request.model_dump_json())

datametric_response = datametrics_instrupy(request)

display(datametric_response.model_dump_json())

#datametric_data = datametric_response.as_dataframe()

#display(datametric_data)

In [ ]:
display(datametrics_response.records[4].model_dump_json())

In [ ]:
display(datametrics_response.records[33].model_dump_json())